In [1]:
import json
import numpy as np
from keras.models import Sequential
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")



In [2]:
DATA_PATH="data.json"
EPOCHS=40
SAVED_MODEL_PATH="model.h5"
BATCH_SIZE=32
PATIENCE=5
LEARNING_RATE=0.0001

In [4]:
def load_dataset(data_path):
    with open (data_path,"r")as fp:
        data=json.load(fp)
    
    #extract input and target
    x=np.array(data["MFCCs"])
    y=np.array(data["labels"])
    print("Training sets loaded!")
    return x,y
    
        
def get_data_splits(data_path,test_size=0.1,test_validation=0.1):
    x,y=load_dataset(data_path)
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=test_size)
    x_train,x_validation,y_train,y_validation=train_test_split(x_train,y_train,test_size=test_validation)
    
   
    #convert input from 2d to 3d array
    x_train=x_train[...,np.newaxis]
    x_validation=x_validation[...,np.newaxis]
    x_test=x_test[...,np.newaxis]
    
    return x_train,x_test,x_validation, y_train, y_test,y_validation



def build_model(input_shape,learning_rate,error="sparse_categorical_crossentropy"):
    #build network
    model=keras.Sequential()
    
    # 1st conv layer
    
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape,
                                      kernel_regularizer=keras.regularizers.l2(0.001)))
    #model.add(Conv2D(8, (3, 3), padding='same',
                 #input_shape=(513, 800, 3)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
    
    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu',
                                     kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
    
    # 3nd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu',
                                     kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))
    
     # flatten output and feed into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    # softmax output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

     #compile the network
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt,loss=error,metrics=["accuracy"])
      
     #print model overview
    model.summary()
   
    return model
    
def main():
        
    #load train ,validation, and test data  splits
    
    x_train, x_validation,x_test,y_train,y_validation,y_test=get_data_splits(DATA_PATH)
    #print(x_train)
    # build the CNN model
    # = x_train.reshape(len(x_train), 1, 1)
    input_shape = (len(x_train),x_train.shape [0]  ,x_train.shape [1])
    
   # print(len(x_train))
   
    
    model = build_model(np.asarray(input_shape), LEARNING_RATE)
    #model = build_model(np.asarray(input_shape), LEARNING_RATE)
   
    # train network
#     model.fit(tf.convert_to_tensor(x_train),tf.convert_to_tensor(y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data= (x_validation, y_validation) )
    model.fit(x_train,y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data= (x_validation, y_validation))


    # evaluate network on test set
    test_error, test_accuracy = model.evaluate(x_test,y_test)
    print(f"Test error: {test_error}, test accuracy: {test_accuracy}")
    
    


    #save the model
    model.save(SAVED_MODEL_PATH)
if __name__ == "__main__":
    main()
                  
                  
    
    
    

Training sets loaded!
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 241, 241, 64)      640       
_________________________________________________________________
batch_normalization_3 (Batch (None, 241, 241, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 121, 121, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 119, 119, 32)      18464     
_________________________________________________________________
batch_normalization_4 (Batch (None, 119, 119, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (No

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).